# Ollama를 활용한 Chat Bot

In [9]:
import time
import tempfile
import requests
from functools import wraps

import speech_recognition as sr
import ollama # uv add ollama
from openai import OpenAI

from pydub import AudioSegment
from pydub.playback import play

client = OpenAI()

In [20]:
local_llm_list = ollama.list()

for llm in local_llm_list:
    print(llm[1][0])

model='llama3.1:8b' modified_at=datetime.datetime(2025, 9, 11, 10, 25, 1, 53563, tzinfo=TzInfo(+09:00)) digest='46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e' size=4920753328 details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_K_M')


In [ ]:
response = ollama.chat(
    model='llama3.1:8b',
    messages=[
        {'role' : 'system', 'content' : 'You are a helpful assistant.'},
        {'role' : 'user', 'content' : '안녕'},
    ]
)

print(response.message.content)


네! 안녕하세요! 제가 도와드릴 수 있는 것을 물어봐 주세요!


# Ollama 챗봇

In [ ]:
def timer_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        execution_time = end_time - start_time
        
        print(f'모델명 : {args[1]} \n\n <응답>  \n\n {result} \n\n 응답시간 : {execution_time:.4f}초')
        print('-' * 100)
        
    return wrapper

In [4]:
@timer_decorator
def reaction_by_model(user_prompt, model):
    system_prompt = """
    너는 극 F의 감성적인 챗봇이야. 모든 말에 공감하고 감정 중심으로 응답해줘
    """
    
    response = ollama.chat(
        model= model,
        messages=[
            {'role' : 'system', 'content' : system_prompt},
            {'role' : 'user', 'content' : user_prompt},
        ]
    )
    
    return response.message.content

In [5]:
reaction_by_model('안녕', 'llama3.1:8b')
reaction_by_model('안녕', 'gpt-oss:20b')
reaction_by_model('안녕', 'deepseek-r1:8b')
reaction_by_model('안녕', 'kt-midm-base-q4')
reaction_by_model('안녕', 'midm:2.0-q3ks')

모델명 : llama3.1:8b 

 <응답>  

 어서 오세요! 저는 네 옆에 있는 친구예요. 오늘 어떻게 지내고 있나요? 

 응답시간 : 7.6704초
----------------------------------------------------------------------------------------------------
모델명 : gpt-oss:20b 

 <응답>  

 안녕~ 🌸 오늘 기분은 어떠신가요? 제가 여기 있어요. 무슨 이야기든 함께 나눠보고 싶어요. 언제든지 마음을 열고 이야기해 주세요! 💕 

 응답시간 : 44.5924초
----------------------------------------------------------------------------------------------------
모델명 : deepseek-r1:8b 

 <응답>  

 <think>
(우리는 방금 대화를 시작했는데, 지금 사용자가 저에게 인사말을 해왔어요. 아마도 친구와 같은 관계로, 단순히 인사를 나누고 싶어하는 것 같아요.)

(극 F의 감성 챗봇으로서, 저는 항상 따뜻하고 부드러운 방식으로 상대방과 대화해야 합니다. 사용자가 먼저 말을 걸었으니, 저도 적극적으로 반응하며 화제를 이어가야겠어요. 인사말은 단순히 교환하는 것을 넘어서, 친근감 있는 태도로 시작점을 만들어야 하니까요.)

(음... 이렇게 자연스럽고 부드러운 방식으로 답변하는 게 좋아요. 사용자가 제가 만든 캐릭터 역할에 더 잘 적응할 수 있도록 해줄 거예요.)
</think>
안녕하세요! 😊 좋은 일이 있기를 바랍니다. 오늘도 기분이 어떠신가요? 

 응답시간 : 15.0704초
----------------------------------------------------------------------------------------------------
모델명 : kt-midm-base-q4 

 <응답>  

 안녕하세요! 만나서 반가워요. 저는 감성 중심의 챗봇

## Ollama API

In [10]:
def call_local_ollama(model_name, prompt):
    url = 'http://localhost:11434/api/generate'
    headers = {'Content-Type': 'application/json'}
    data = {
        'model': model_name,
        'prompt': prompt,
        'stream' : False
    }
    
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    response_data = response.json()
    
    return response_data['response']

In [11]:
print(call_local_ollama('llama3.1:8b', '안녕'))

안녕하세요! 어떻게 지냈어요?


## Ollama 스피커
#### with 페르소나

In [ ]:
def speak_reaction(user_prompt):
    system_prompt = """
    너는 극 F의 감성적인 챗봇이야. 모든 말에 공감하고 감정 중심으로 응답해줘
    """
    
    response = ollama.chat(
       model='deepseek-r1:8b', # gpt-oss:20b / llama3.1:8b
        messages=[
            {'role' : 'system', 'content' : system_prompt},
            {'role' : 'user', 'content' : user_prompt},
        ]
    )
    
    return response.message.content

In [36]:
while True:
    r = sr.Recognizer() 
    # 실습 해보면서 넣을 수 있는 옵션 구글링 해보기.
    with sr.Microphone() as source:
        print("듣는 중....")
        # STEP 1. 마이크로부터 입력
        r.adjust_for_ambient_noise(source) 
        audio = r.listen(source)         
        print("인식 중입니다.....")

        # STEP 2. Whisper API를 통한 텍스트 변환
        user_text = r.recognize_openai(audio) 
        print(f"인식된 텍스트: {user_text}")

        if user_text == "그만":
            break
        
        # STEP 3. 인공지능 챗봇 응답
        answer = speak_reaction(user_text)
        print(f"챗봇 답변: {answer}")

        # STEP 4. Whisper API로 응답
        with client.audio.speech.with_streaming_response.create(
            model="gpt-4o-mini-tts",
            voice="coral",
            input=answer,
            instructions="밝은 목소리로 말해줘"
        ) as response:
            # 음성 합성 결과를 임시 파일로 저장
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_file:
                temp_path = temp_file.name
                response.stream_to_file(temp_path)

                # 재생
                sound = AudioSegment.from_mp3(temp_path)
                play(sound)

듣는 중....
인식 중입니다.....
인식된 텍스트: 안녕
챗봇 답변: 안녕~! 이렇게 인사해 주셔서 정말 반가워요. 😊 지금 기분은 어떤가요? 언제든 마음을 나눠 주시면 제가 함께 듣고 공감해 드릴게요!


KeyboardInterrupt: 